<h1>Spaceship Titanic Competion</h1>

<h2>Import Dependencies</h2>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing

<h2>Import Data</h2>

In [ ]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
sub_df = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

<h2>Quick Look at Data</h2>

In [ ]:
# Look at the first entries
# Uncomment the dataframe you want to check

train_df.head()
#test_df.head()
#sub_df.head()

In [ ]:
# Description of the data
train_df.describe()

In [ ]:
# Shapes of the dataframes
train_df.shape, test_df.shape, sub_df.shape

<h2>Data Simple Preparation</h2>

In [ ]:
# Change the label from boolean to numeric (0 or 1)
train_df['Transported'] = train_df['Transported'].astype(int)

In [ ]:
df = pd.concat([train_df,test_df], axis = 0) #For easy preprocessing, it is concatenated the train and test data dataframes
df.describe()

In [ ]:
# More info of the data
df.info()

In [ ]:
# Simple function to expose clearer the info about missing and uniqueness of entries in the data
def summary(df):
    print(f"Dataset has {df.shape[1]} features and {df.shape[0]} examples.")
    summary = pd.DataFrame(index=df.columns)
    summary["Unique"] = df.nunique().values
    summary["Missing"] = df.isnull().sum().values
    summary["Duplicated"] = df.duplicated().sum()
    summary["Types"] = df.dtypes
    return summary

summary(train_df)

In [ ]:
# Distribution of the labels in the training data
train_df.Transported.value_counts().plot(kind = 'pie', labels = ['True','False'], autopct='%.0f%%')

<h2>Data Visualization</h2>

After gathering the data and examining it broadly, let's visualize some of the features and their relation to the final label 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sns.countplot(data = train_df, x = 'HomePlanet', hue = 'Transported')

<h3>Insights</h3>
<ul>
    <li>Rate of transportation in Europa is way higher than the other 2</li>
    <li>Rate of transportation in Earth the lowest, and below 50%</li>
</ul>

In [ ]:
sns.countplot(data = train_df, x = 'Destination', hue = 'Transported')

<h3>Insights</h3>
<ul>
    <li>With "55 Cancri e" as a destination is the only one with a rate of transportation over 50%</li>
    <li>Not much more can be concluded from this graph</li>
</ul>

Let's now take a look the the combinations of Home Planet/Destination and the transportation information

In [ ]:
pd.DataFrame(train_df.groupby(['HomePlanet','Destination','Transported']).size())

<h3>Insights</h3>
<ul>
    <li>It is possible to verify that highest number of travels are between Earth and TRAPPIST-1e</li>
    <li>It is possible to see that highest transportation rate travel is Europa->55 Cancri e (68.96%)</li>
</ul>

In [ ]:
# Box plot for the age distribition in both Transported and Non-Transported labeles examples
px.box(train_df, x = 'Transported', y = 'Age')

<h3>Insights</h3>
<ul>
    <li>Similar age distributions for Transported and Non-Transported</li>
    <li>No relation between age and label. Maybe in some other way, like age block, age data can be relevant</li>
</ul>

In [ ]:
numeric_cols = train_df.select_dtypes(include = 'number').columns.tolist()
sns.pairplot(train_df[numeric_cols])

<h3>Insights</h3>
<ul>
    <li>Possible to see that those that spent more on food had an higher count on Transported</li>
    <li>Possible to see that those that spent more in VRDeck and Spa had an lower count on Transportedr</li>
</ul>

In [ ]:
pivot = pd.pivot_table(train_df, values = 'Transported', index=['VIP','CryoSleep'], columns=train_df['Transported'], aggfunc=len)
pivot = pd.pivot_table(train_df, values = 'Transported', index=['CryoSleep','VIP'], columns=train_df['Transported'], aggfunc=len)
pivot
#pd.DataFrame(train_df.groupby(['VIP','CryoSleep','Transported']).size())

<h3>Insights</h3>
<ul>
    <li>When the VIP and CrySleep usage is combined, the label is always Transported.</li>
    <li>Much more examples where neither VIP or CryoSleep were used. Low VIPs.</li>
    <li>Being VIP doesn't easily transpose into Transported label, if anything it may lower the chances.</li>
    <li>Looks like CryoSleep usage has greater effect into the label being Transported. Probably good column of the dataset.</li>
</ul>

In [ ]:
# Additional data visualization to prove that CryoSleep is a good column in the dataset
pivot1 = pd.pivot_table(train_df, values = 'Transported', index=['CryoSleep'], columns=train_df['Transported'], aggfunc=len)
sns.heatmap(pivot1, annot = True, cbar = True, fmt = "d")

<h3>Insights</h3>
<ul>
    <li>Clearly, it is possible to see that when CryoSleep is used, the chances of being transported increase.</li>
    <li>Of those were transported, around 58% were in CryoSleep. Of those that used CryoSleep, around 80% were Transported.</li>
</ul>